In [335]:
import pandas as pd
from collections import Counter
from icecream import ic

In [336]:
file = "test.dat"
file = "day.dat"
lines = open(file,'r').read().split('\n')
columns = ['hand','bid']
df = pd.DataFrame([s.split(" ") for s in lines], columns=columns)
df.head()


,hand,bid
0,TQA26,14
1,66656,612
2,JK3JT,442
3,A6QQA,214
4,Q3777,131


In [337]:
hand_type = {7: 'Five of a kind', 6: 'Four of a kind', 5: 'Full house', 
             4: 'Three of a kind', 3: 'Two pair', 2: 'One pair', 1: 'High card'}
df['count'] = [list(dict(Counter(hand).most_common(2)).values()) for hand in df['hand']]
for row in df.itertuples():
    if 5 in row.count or row.count[0] >= 5:
        df.at[row.Index, 'hand_type'] = 7
        df.at[row.Index, 'hand_type_text'] = hand_type[7]
    elif 4 in row.count:
        df.at[row.Index, 'hand_type'] = 6
        df.at[row.Index, 'hand_type_text'] = hand_type[6]
    elif 3 in row.count and 2 in row.count:
        df.at[row.Index, 'hand_type'] = 5
        df.at[row.Index, 'hand_type_text'] = hand_type[5]
    elif 3 in row.count:
        df.at[row.Index, 'hand_type'] = 4
        df.at[row.Index, 'hand_type_text'] = hand_type[4]
    elif row.count == [2,2]:
        df.at[row.Index, 'hand_type'] = 3
        df.at[row.Index, 'hand_type_text'] = hand_type[3]
    elif 2 in row.count:
        df.at[row.Index, 'hand_type'] = 2
        df.at[row.Index, 'hand_type_text'] = hand_type[2]
    else:
        df.at[row.Index, 'hand_type'] = 1
        df.at[row.Index, 'hand_type_text'] = hand_type[1]


In [338]:
score_face = {'A': 14, 'K': 13, 'Q': 12, 'J': 11, 'T': 10, '9': 9, '8': 8, '7': 7, '6': 6, '5': 5, '4': 4, '3': 3, '2': 2}
df['score_card'] = [[score_face.get(face, face) for face in hand] for hand in df['hand']]
df[['card1','card2','card3','card4','card5']] = pd.DataFrame(df['score_card'].to_list(), index=df.index)

In [339]:
df.sort_values(by=['hand_type','card1','card2', 'card3', 'card4', 'card5'], inplace=True, 
               ascending=True, ignore_index=True)
df['score'] = (df.index + 1) * df['bid'].astype(int)
df

,hand,bid,count,hand_type,hand_type_text,score_card,card1,card2,card3,card4,card5,score
0,254JK,113,"[1, 1]",1.0,High card,"[2, 5, 4, 11, 13]",2,5,4,11,13,113
1,254KA,947,"[1, 1]",1.0,High card,"[2, 5, 4, 13, 14]",2,5,4,13,14,1894
2,274Q5,672,"[1, 1]",1.0,High card,"[2, 7, 4, 12, 5]",2,7,4,12,5,2016
3,276Q5,359,"[1, 1]",1.0,High card,"[2, 7, 6, 12, 5]",2,7,6,12,5,1436
4,2786T,391,"[1, 1]",1.0,High card,"[2, 7, 8, 6, 10]",2,7,8,6,10,1955
...,...,...,...,...,...,...,...,...,...,...,...,...
995,AATAA,508,"[4, 1]",6.0,Four of a kind,"[14, 14, 10, 14, 14]",14,14,10,14,14,505968
996,AAQAA,102,"[4, 1]",6.0,Four of a kind,"[14, 14, 12, 14, 14]",14,14,12,14,14,101694
997,AAA8A,166,"[4, 1]",6.0,Four of a kind,"[14, 14, 14, 8, 14]",14,14,14,8,14,165668
998,AAAKA,533,"[4, 1]",6.0,Four of a kind,"[14, 14, 14, 13, 14]",14,14,14,13,14,532467


In [340]:
# 253979109 is too high
# answer is 253603890
df['score'].sum()

253603890

Part 2

Ended up figuring out how the number of jokers affects each hand, by doing the calculation by hand.
Got it wrong several times and sought help on the almight Reddit page. So it goes.
```
				jokers
		index	
			1 	2	3	4	5			
high card	1	2	-	-	-	-
one pair	2	4	4	-	-	-
two pair	3	5	6	-	-	-
three kind	4	6	-	6	-	-
full house	5	-	7	7	-	-
four kind	6	7	-	-	7	-
five kind	7	-	-	-	-	-
```

In [344]:
df['jacks'] = [hand.count('J') for hand in df['hand']]
mapping_table = {(1,1):2, 
                 (2,1):4, (2,2):4, 
                 (3,1):5, (3,2):6,
                 (4,1):6, (4,3):6,
                 (5,2):7, (5,3):7, 
                 (6,1):7, (6,4):7,}
def map_hand_wild(row):
    key = (row['hand_type'], row['jacks'])
    if key in mapping_table:
        return mapping_table[key]
    else:
        return row['hand_type'] 

df['hand_wild'] = df.apply(map_hand_wild, axis=1)
df['hand_wild_text'] = df['hand_wild'].map(hand_type)
score_wild = {'A': 14, 'K': 13, 'Q': 12, 'J': 1, 'T': 10, '9': 9, '8': 8, '7': 7, '6': 6, '5': 5, '4': 4, '3': 3, '2': 2}
df['score_card_wild'] = [[score_wild.get(face, face) for face in hand] for hand in df['hand']]
df[['card1w','card2w','card3w','card4w','card5w']] = pd.DataFrame(df['score_card_wild'].to_list(), index=df.index)
df.sort_values(by=['hand_wild','card1w','card2w', 'card3w', 'card4w', 'card5w'], inplace=True, 
               ascending=True, ignore_index=True)
df['score_wild'] = (df.index + 1) * df['bid'].astype(int)
df[df['jacks'] > 0][['hand', 'hand_type', 'hand_type_text', 'hand_wild', 'hand_wild_text', 'jacks']].sample(10)

,hand,hand_type,hand_type_text,hand_wild,hand_wild_text,jacks
814,JKK4K,4.0,Three of a kind,6.0,Four of a kind,1
943,QQ7QJ,4.0,Three of a kind,6.0,Four of a kind,1
447,J7552,2.0,One pair,4.0,Three of a kind,1
475,2T25J,2.0,One pair,4.0,Three of a kind,1
647,K5J9K,2.0,One pair,4.0,Three of a kind,1
580,855JK,2.0,One pair,4.0,Three of a kind,1
249,95JA8,1.0,High card,2.0,One pair,1
604,96J94,2.0,One pair,4.0,Three of a kind,1
805,JJK2K,3.0,Two pair,6.0,Four of a kind,2
746,88J99,3.0,Two pair,5.0,Full house,1


In [342]:
# test is 5905
# 254085991 is too high
# 254087693 is too high
# 254569236 is too high
# 252847906 is incorrect
# 253688431 is incorrect
# 252972326 is incorrect
# 253630098 is correct
df['score_wild'].sum()

253630098